In [1]:
import platform

platform.python_version()

'3.11.3'

In [2]:
import concurrent
import concurrent.futures
import time
import math

## 1. 락처리 

## 1-1 스레드 락처리 

- concurrent.futures 모듈은 스레드나 프로세스 기반의 비동기 작업을 처리할 때 사용됩니다. 
- 이 모듈은 일반적으로 ThreadPoolExecutor 클래스를 사용하여 작업을 실행하고 관리합니다.

- 여기에는 Lock 클래스가 없습니다. 대신, 스레드나 프로세스 간의 상호배제를 위해 파이썬의 표준 라이브러리인 threading 모듈의 Lock 클래스를 사용할 수 있습니다.

In [3]:
import concurrent.futures
import threading

# 공유 변수
shared_var = 0

# 락 생성
lock = threading.Lock()

# 공유 변수를 증가시키는 함수
def increment_shared_var():
    global shared_var
    for _ in range(100000):
        with lock:  # 락을 사용하여 공유 변수에 대한 안전한 업데이트 보장
            shared_var += 1

# 스레드 풀 생성
with concurrent.futures.ThreadPoolExecutor() as executor:
    # 스레드를 풀에 추가
    futures = [executor.submit(increment_shared_var) for _ in range(10)]

# 모든 스레드의 작업이 완료될 때까지 대기
concurrent.futures.wait(futures)

# 공유 변수 값 출력
print("Shared variable value:", shared_var)


Shared variable value: 1000000


## 1-2 멀티프로세스 락 처리 

### concurrent.futures 모듈
- 스레드와 프로세스 기반의 비동기 작업을 처리하는 데 사용됩니다. 
- 여기서는 스레드 풀 및 프로세스 풀을 사용하여 작업을 관리합니다. 
- 이 모듈은 기본적으로 스레드 기반 및 프로세스 기반의 비동기 실행을 추상화하고,  ProcessPoolExecutor 클래스를 제공하여 관리를 용이하게 합니다.

- 일반적으로 concurrent.futures 모듈에서는  multiprocessing.Lock을 사용하여 프로세스 간의 상호 배제를 달성합니다.

In [7]:
%%writefile prcess_lock4.py

import concurrent.futures
import threading
import multiprocessing

# 공유 변수
shared_var = 0

# 스레드 락
thread_lock = threading.Lock()

# 프로세스 락
process_lock = multiprocessing.Lock()

# 공유 변수를 증가시키는 함수
def increment_shared_var_thread():
    global shared_var
    for _ in range(100000):
        with thread_lock:
            shared_var += 1

def increment_shared_var_process():
    global shared_var
    for _ in range(100000):
        with process_lock:
            shared_var += 1
            
if __name__ == "__main__" : 

    # 스레드 풀 생성
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # 스레드를 풀에 추가
        futures = [executor.submit(increment_shared_var_thread) for _ in range(10)]

    # 모든 스레드의 작업이 완료될 때까지 대기
    concurrent.futures.wait(futures)

    # 공유 변수 값 출력
    print("Shared variable value (threading):", shared_var)

    # 프로세스 풀 생성
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # 프로세스를 풀에 추가
        futures = [executor.submit(increment_shared_var_process) for _ in range(10)]

    # 모든 프로세스의 작업이 완료될 때까지 대기
    concurrent.futures.wait(futures)

    # 공유 변수 값 출력
    print("Shared variable value (multiprocessing):", shared_var)


Overwriting prcess_lock4.py


In [8]:
!python prcess_lock4.py

Shared variable value (threading): 1000000
Shared variable value (multiprocessing): 1000000
